# Apply trace metal concentrations to river classification

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean

%matplotlib inline

## River categories:

1. Glaciers
2. Continental
3. Other

In [3]:
rclass = nc.Dataset('/ocean/brogalla/GEOTRACES/data/river_class-pb-scavenging.nc','r')
river_class = rclass.variables['rclass']
river_class = np.array(river_class)

### Apply trace metal concentrations

Notebook to find these concentratons: /analysis-brogalla/forcing/rivers/find_characteristic_tracer_concentrations.ipynb

In [7]:
SPM_rivers = np.zeros(river_class.shape)
dMn_rivers = np.zeros(river_class.shape)

In [8]:
multiplication_factor_glacial     = 1
multiplication_factor_continental = 1

SPM_factor = 1

Manganese:

In [9]:
dMn_rivers[river_class==1.0] = 164e-9*multiplication_factor_glacial # glaciers
dMn_rivers[river_class==2.0] = 29.8e-9*multiplication_factor_continental  # continental
dMn_rivers[river_class==3.0] = 2.2e-9   # other
dMn_rivers[river_class==4.0] = 2.2e-9   # Gulf of Boothia

In [10]:
SPM_rivers[river_class==1.0] = SPM_factor*261e-6*multiplication_factor_glacial # glaciers
SPM_rivers[river_class==2.0] = SPM_factor*1.196e-5*multiplication_factor_continental  # continental
SPM_rivers[river_class==3.0] = SPM_factor*4.08e-6   # other
SPM_rivers[river_class==4.0] = SPM_factor*4.08e-6   # Gulf of Boothia

##### Write to NetCDF file

In [11]:
c = nc.Dataset('/ocean/brogalla/GEOTRACES/data/runoff/ANHA12_runoff_monthly_combined_Dai_Trenberth_Bamber_y2002.nc','r')
lon_rf = np.array(c.variables['nav_lon'])
lat_rf = np.array(c.variables['nav_lat'])

In [12]:
ncd = nc.Dataset('/ocean/brogalla/GEOTRACES/data/mn-river-forcing-base-extended.nc', 'w', zlib=True)
ncd.createDimension('x',len(c.dimensions['x']))
ncd.createDimension('y',len(c.dimensions['y']))

# variables
nav_lat = ncd.createVariable('nav_lat', 'float32', ('y','x'))
nav_lat.long_name = 'Latitude'
nav_lat.units = 'degrees_north'
nav_lat[:] = lat_rf

nav_lon = ncd.createVariable('nav_lon', 'float32', ('y','x'))
nav_lon.long_name = 'Longitude'
nav_lon.units = 'degrees_east'
nav_lon[:] = lon_rf

spm_rivers = ncd.createVariable('spm_rivers', 'float32', ('y','x'))
spm_rivers.units = 'kg/L'
spm_rivers.long_name = 'Suspended Particulate Matter content'  
spm_rivers.coordinates = 'nav_lon nav_lat'
spm_rivers[:] = SPM_rivers

dmn_rivers = ncd.createVariable('dmn_rivers', 'float32', ('y','x'))
dmn_rivers.units = 'mol/L'
dmn_rivers.long_name = 'Dissolved Mn concentration'  
dmn_rivers.coordinates = 'nav_lon nav_lat'
dmn_rivers[:] = dMn_rivers

ncd.close()

Check files:

In [13]:
file = nc.Dataset('/ocean/brogalla/GEOTRACES/data/mn-river-forcing-base-extended.nc')

dmn = np.array(file.variables['dmn_rivers'])
spm = np.array(file.variables['spm_rivers'])

In [14]:
print('dMn class 1: %.2e' % np.average(dmn[river_class == 1]))
print('dMn class 2: %.2e' % np.average(dmn[river_class == 2]))
print('dMn class 3: %.2e' % np.average(dmn[river_class == 3]))
print('------------------')
print('pMn class 1: %.2e' % np.average(spm[river_class == 1]))
print('pMn class 2: %.2e' % np.average(spm[river_class == 2]))
print('pMn class 3: %.2e' % np.average(spm[river_class == 3]))

dMn class 1: 1.64e-07
dMn class 2: 2.98e-08
dMn class 3: 2.20e-09
------------------
pMn class 1: 2.61e-04
pMn class 2: 1.20e-05
pMn class 3: 4.08e-06
